In [ ]:
from ckiptagger import data_utils, construct_dictionary

In [2]:
import tensorflow as tf
import keras

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc


Using TensorFlow backend.


In [138]:
def word_proc(data_path,column_name,
              tt_begin = 0,tt_end = 50,
              output_name="df_test_out.csv"):
    """
        data_path(str):資料路徑(字串)，輸入的資料應為CSV檔，ex:
        column_name(str):要處理的欄位名稱
        tt_begin(int):起始列(整數)
        tt_end(int):結束列(整數)
        output_CSV_name:輸出的檔案名稱，指定為CSV檔    
    """
    from ckiptagger import WS, POS, NER
    
    file_type = data_path[-4:]
    print("輸入檔案格式為 {}".format(file_type))
    if 'csv' in file_type:
        data = pd.read_csv(data_path)
    elif 'xls' in file_type:
        data = pd.read_excel(data_path)
    else:
        #print("檔案須為CSV或XLS/XLSX")
        raise Exception("檔案須為CSV或XLS/XLSX")
        #return None
    
    
    text = data[column_name]

    ls_Test = list(text[(tt_begin-1):tt_end])
    print("Got list\n 載入 WS 資料...")
    
    ws = WS("./data")
    print("============================\n 開始執行 WS\n============================\n")
    word_sentence_list = ws(ls_Test
                                # sentence_segmentation=True, # To consider delimiters
                                # segment_delimiter_set = {",", "。", ":", "?", "!", ";"}),
                                # This is the defualt set of delimiters
                                # recommend_dictionary = dictionary1, # words in this dictionary are encouraged
                                # coerce_dictionary = dictionary2, # words in this dictionary are forced
                            )
    del ws
    del WS
    gc.collect()
    print("============================\n WS 完成\n============================\n載入 POS 資料...")
    
    print("============================\n 開始執行 POS \n============================\n")
    pos = POS("./data")
    pos_sentence_list = pos(word_sentence_list)
    del pos
    del POS
    gc.collect()
    print("============================\n POS 完成\n============================\n載入NER 資料...")
    
    
    print("============================\n 開始執行 NER \n============================\n")
    ner = NER("./data")
    entity_sentence_list = ner(word_sentence_list, pos_sentence_list)
    del NER
    del ner
    gc.collect()
    print("============================\n NER 完成\n============================\n")
        
    df_Test = pd.DataFrame(ls_Test,columns=["Text"])
    df_Test['WS'] = np.array(word_sentence_list)
    df_Test['POS'] = np.array(pos_sentence_list)
    df_Test['NER'] = np.array(entity_sentence_list)
    print("============================\n Dataframe Created \n============================\n")
        
    df_Test.to_csv(output_name)
    return df_Test,word_sentence_list,pos_sentence_list,entity_sentence_list

In [139]:
data_path_a = "KCC_Data/KCCNews11554.csv"
data_path_c = 'KCC_Data/20160419-20190804-自由(985筆).xlsx'
column_name_a = "Text"
column_name_c = "內容"
output_name_a = "mod_test.csv"

In [140]:
demo_df,ws_list,pos_list,ner_list = word_proc(data_path = data_path_c,
          column_name = column_name_c,
          tt_begin = 1,tt_end = 10,
          output_name=output_name_a)

輸入檔案格式為 xlsx
Got list
 載入 WS 資料...
 開始執行 WS

 WS 完成
載入 POS 資料...
 開始執行 POS 

 POS 完成
載入NER 資料...
 開始執行 NER 

 NER 完成

 Dataframe Created 



In [142]:
len(ner_list),type(ner_list[0]),len(ner_list[9])

(10, set, 75)

In [78]:
#type(ner_list),len(ner_list),ner_list[:2]

In [119]:
#type(ner_list[0]),len(ner_list[0]),len(ner_list[1]),ner_list[0]

In [120]:
#ner_list[0]

In [24]:
#new_list = ner_list[0:10].copy()

In [25]:
#del class_list
#del ner_list


# 1.取出NER類別及專有名詞，分別存到df_class_list 及df_ner_list中
# 2.將其組成DataFrame['cl','NER']；欄位'cl' 為專有名詞類別、欄位'NER'為專有名詞

In [143]:
df_class_list = list()
df_ner_list = list()

count = 0
for i in range(len(ner_list)):
    for j in range(len(ner_list[i])):
        count = count +1
        a = ner_list[i].pop()
        df_class_list.append(a[2])
        df_ner_list.append(a[3])
        #print(type(new_list[j]),(new_list[j]))
print(count)

364


In [144]:
extract_df = pd.DataFrame(
{'cl':df_class_list,
 'NER':df_ner_list
})

In [145]:
print(extract_df.describe())
extract_df.groupby('cl').count()

            cl  NER
count      364  364
unique      12  172
top     PERSON  韓國瑜
freq       107   16


,NER
cl,
CARDINAL,37
DATE,62
FAC,7
GPE,55
LANGUAGE,1
LOC,5
MONEY,12
NORP,3
ORDINAL,4


In [69]:
#測試用
#copy_df = test_df.copy()

In [43]:
#len(copy_df),copy_df['cl'].unique()

(1139,
 array(['LOC', 'DATE', 'FAC', 'GPE', 'PERCENT', 'PERSON', 'ORG', 'ORDINAL',
        'MONEY', 'TIME', 'NORP', 'CARDINAL', 'LANGUAGE', 'LAW', 'EVENT',
        'QUANTITY'], dtype=object))

In [71]:
#test_df

In [146]:
dd_extract_df = extract_df.drop_duplicates()
len(extract_df),len(dd_extract_df)

(364, 177)

In [147]:
dd_extract_df['cl'].unique()

array(['GPE', 'ORG', 'DATE', 'TIME', 'CARDINAL', 'PERSON', 'MONEY', 'FAC',
       'NORP', 'LOC', 'LANGUAGE', 'ORDINAL'], dtype=object)

In [148]:
df_exp = dd_extract_df.loc[dd_extract_df['cl'].isin(['LOC','PERSON', 'ORG', 'LAW', 'EVENT','GPE'])]
len(df_exp)
#https://stackoverflow.com/questions/17071871/how-to-select-rows-from-a-dataframe-based-on-column-values

101

In [149]:
df_exp_2 = df_exp[df_exp['NER'].map(len) >= 2].copy()
len(df_exp_2)

96

In [150]:
df_exp_2[:5].sort_values(by=['cl'])

,cl,NER
0,GPE,高市路竹區頂寮里
22,GPE,高雄
1,ORG,水利局
5,ORG,民進黨
10,PERSON,邱志偉


In [151]:
df_exp_2.to_csv('NERrlt_20160419-20190804-自由(985筆)_1_9.csv')

In [103]:
#print(new_list[0])
s = new_list[0]
a = s.pop()
print(a)
print(a[2])
print(a[3])
class_list.append(a[2])
ner_list.append(a[3])
new_list[0]
class_list,ner_list

(38, 40, 'PERSON', '阿扁')
PERSON
阿扁


(['GPE',
  'ORG',
  'PERSON',
  'PERSON',
  'ORG',
  'PERSON',
  'CARDINAL',
  'ORG',
  'PERSON'],
 ['高雄', '親民黨', '馬英九', '陳水扁', '民進黨團', '馬', '一', '親民黨', '阿扁'])

In [96]:
e = iter(s)
e

In [20]:
a[3]

'親民黨'

In [25]:
df_NER = pd.DataFrame(columns=['Class', 'NER'])

In [26]:
df_NER

,Class,NER


In [116]:
len(class_list),len(ner_list)

(394, 394)

In [129]:
del test_df

In [99]:
data_path_a = "KCC_Data/KCCNews11554.csv"
data_path_b = "KCC_Data/KCCNews11554.xls"

In [103]:
if 'csv' in data_path_a[-4:]:
    print("YES")

YES
